<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #4: Pre-Procesamiento de textos y Featuring Engineering 

### Gabriel Cruz Parra - Código: 617202013
`Fecha de entrega: Marzo 18, 2021. (Antes del inicio de la próxima clase).`

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import re
import pandas as pd
from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords_sp = stopwords.words('spanish')

from sklearn.feature_extraction.text import CountVectorizer

# Punto 1: Pre-Procesamiento

- `[5 pts]` Leer el archivo `dialogos.csv` usando `pandas`
- `[10 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números y sin palabras vacias
- `[7 pts]` Usando la nueva columna, ¿cuáles son las 10 palabras más usadas? 

In [6]:
dialogos = pd.read_csv("./SampleFiles/dialogos.csv", sep=",", encoding="UTF-8")
dialogos

,Locución
0,Francamente no
1,"Lo del canadiense. Por favor, como que vosotro..."
2,Tienes razón. A lo mejor así te liberas de tu ...
3,"Pues, tú sabrás"
4,Para mí que fue Krieger
...,...
954,"Yo qué sé, digo yo qué será"
955,Lo han entendido mal. Las joyas que venden en ...
956,Oooo... Me llamo Vivian
957,¿De dónde? ¿Del coño de tu madre?


In [7]:
# creación paso a minúsculas
dialogos["pre-procesado"] = dialogos["Locución"].str.lower()
# retirar caracteres especiales y números
dialogos["pre-procesado"] = dialogos["pre-procesado"].apply(lambda x : re.sub(r"[\W\d]", " ", x))
# retirar palabras vacías (stop words)
dialogos["pre-procesado"] = dialogos["pre-procesado"].apply(lambda x : " ".join([w for w in word_tokenize(x) if not w in stopwords_sp]))
# Mostrar el dataframe
dialogos

,Locución,pre-procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger
...,...,...
954,"Yo qué sé, digo yo qué será",sé digo
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre


In [8]:
#definir función para obtener frecuencias de palabras
def obtenerFrecuenciasPalabras(columna):
    misFrecuencias = {}
    # recorrer cada valor de la columna del dataframe
    for valor in columna:
        #recorrer cada palabra "tokenizada"
        for key in word_tokenize(valor):
            # preguntar si existe la palabra en diccionario, si existe sumar 1, sino asignar el valor 1
            if key in misFrecuencias:
                misFrecuencias[key]= misFrecuencias.get(key)+1
            else:
                misFrecuencias[key]=1            
    return misFrecuencias

In [9]:
# definir función para imprimir las n palabras mas usadas
def imprimirDiezPalabrasMasUsadas(frecuencias, n):
    i = 0
    # obtener una lista de tuplas con el diccionario ordenado
    topTen = sorted(frecuenciasProcesadas.items(), key=lambda x: x[1], reverse=True) 
    for  val in topTen:
        # imprimir llave (palabra) y valor (frecuencia)
        print(f"Palabra {i+1}. {val[0]}, cantidad: {val[1]}")
        i+=1
        # preguntar si ha impreso las n primeras palabras, si es así terminar el ciclo
        if(i==n):
            break
    

In [10]:
# ejecución de las funciones para obtener resultado
frecuenciasProcesadas = obtenerFrecuenciasPalabras(dialogos["pre-procesado"])
imprimirDiezPalabrasMasUsadas(frecuenciasProcesadas, 10)


Palabra 1. bien, cantidad: 36
Palabra 2. si, cantidad: 34
Palabra 3. cómo, cantidad: 34
Palabra 4. así, cantidad: 30
Palabra 5. oh, cantidad: 28
Palabra 6. vamos, cantidad: 26
Palabra 7. sarcasmobol, cantidad: 24
Palabra 8. aquí, cantidad: 24
Palabra 9. bueno, cantidad: 24
Palabra 10. lana, cantidad: 22


In [21]:
# Otra solución más simple con librería collections función objeto Counter
conteos= Counter(dialogos["pre-procesado"].str.cat(sep=" ").split())
i=1
# Recorrer el listado de tuplas para impresión en pantalla más amable
for val in conteos.most_common(10):
    print(f"palabra {i}. {val[0]}, cantidad: {val[1]}")
    i+=1

palabra 1. bien, cantidad: 36
palabra 2. si, cantidad: 34
palabra 3. cómo, cantidad: 34
palabra 4. así, cantidad: 30
palabra 5. oh, cantidad: 28
palabra 6. vamos, cantidad: 26
palabra 7. sarcasmobol, cantidad: 24
palabra 8. aquí, cantidad: 24
palabra 9. bueno, cantidad: 24
palabra 10. lana, cantidad: 22


# Punto 2: Representación vectorial

- `[15 pts]` Crear una bolsa de palabras (BoW) del corpus usando la columna pre-procesada (Usando la función de `sklearn`)
- `[10 pts]` ¿Cuántas palabras hay en el vocabulario? (Usando la función de `sklearn`)

In [12]:
# Creación de bolsa de palabras en el dataframe de diálogos con librería scikit learn, función objeto CountVectorizer
countVect = CountVectorizer()
bowRep = countVect.fit_transform(dialogos["pre-procesado"])
dialogos["bow"] = [row for row in bowRep.toarray()]
dialogos


,Locución,pre-procesado,bow
0,Francamente no,francamente,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Pues, tú sabrás",pues sabrás,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Para mí que fue Krieger,krieger,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
print("Cantidad de palabras: ", len(countVect.get_feature_names()))

Vocabulario:  2115


# Punto 3: 🤔

- `[3pts]` ¿Cuándo es una mejor idea usar una bolsa de n-gramas en vez de una bolsa de palabras?
- **Respuesta:** Es mejor utilizar utilizar una bolsa de n-gramas cuando es necesario tener en consideración el orden de aparición de las palabras en una oración o secuencia de palabras (es importante conservar el contexto).  

# BONUS Punto 4: 😜

- `[2pt]` ¿A qué pertenecen los dialogos de ese archivo? 

- **Respuesta:** Los diálogos pertenecen a la serie animada South Park; temporada 16, episodio 8.